In [103]:
import random

ModuleNotFoundError: No module named 'distutils'

In [104]:
import json

def read_graph(path: str): # return graph, max_install_each_node, max_latency, max_cost_active, max_cost_install, list_server_node (đọc đồ thị )
    try:
        file = open(path, "r")
        first_line = file.readline().split(" ")
        num_vnf = int(first_line[0])
        max_install_each_node = int(first_line[1])
        second_line = int(file.readline())
        set_node = {}
        list_server_node = []
        max_latency = 0
        max_cost_active = 0
        max_cost_install = 0
        for i in range(second_line):
            line = file.readline().replace("\n", "").split(" ")
            description = {}
            if int(line[2]) != -1:
                description["delay"] = int(line[1])
                max_latency += int(line[1])
                description["active_cost"] = int(line[2])
                max_cost_active += int(line[2])

                list_cosst = []
                for j in range(num_vnf):
                    list_cosst.append(int(line[3 + j]))
                    max_cost_install += int(line[3 + j])
                description["list_install_cost"] = list_cosst
                set_node.update({int(line[0]): description})
            else:
                description["delay"] = int(line[1])
                set_node.update({int(line[0]): description})
        num_edges = int(file.readline())

        for i in range(num_edges):
            line = file.readline().replace("\n", "").split(" ")
            description_node1 = set_node.get(int(line[0]), {})
            description_node2 = set_node.get(int(line[1]), {})
            neighbor1 = description_node1.get("neighbor", {})
            neighbor2 = description_node2.get("neighbor", {})
            neighbor1.update({int(line[1]): int(line[2])})
            neighbor2.update({int(line[0]): int(line[2])})
            max_latency += int(line[2])
            description_node1.update({"neighbor": neighbor1})
            description_node2.update({"neighbor": neighbor2})
            set_node.update({int(line[0]): description_node1})
            set_node.update({int(line[1]): description_node2})
            graphviz_string += str(line[0]) + " -- " + str(line[1]) + ";\n"
        file.close()
        with open("graph.json", "w") as outfile:
            json.dump(set_node, outfile, indent=4)

        # graphviz_string = "graph G {\n"
        # for i in range(len(set_node)):
        #     if set_node[i].get("is_server", False):
        #         graphviz_string += str(i) + " [color=red, style=filled];\n"
        #     else:
        #         graphviz_string += str(i) + ";\n"
        # for i in range(len(set_node)):
        #     neighbor = set_node[i].get("neighbor", {})
        #     for j in neighbor:
        #         graphviz_string += str(i) + " -- " + str(j) +";\n"
        graphviz_string += "}"
        with open("input.dot", "w") as outfile:
            outfile.write(graphviz_string)
        return set_node, max_install_each_node, max_latency, max_cost_active, max_cost_install, list_server_node
    except Exception as e:
        print(e)
        return None


In [105]:
graph, max_install_each_node, max_latency, max_cost_active, max_cost_install, list_server_node = read_graph("input.txt")
max_latency

44867

In [106]:
def read_request(path:str="request10.txt"):
    try:
        file = open(path, "r")

        nums_request = int(file.readline())
        list_request = []
        for i in range(nums_request):
            line = file.readline().replace("\n", "").split(" ")
            description = {}
            description['memory'] = int(line[0])
            description['bandwidth'] = int(line[1])
            description['cpu'] = int(line[2])
            description['source'] = int(line[3])
            description['destination'] = int(line[4])
            vnfs_chain = []
            for i in range (5, len(line)):
                vnfs_chain.append(int(line[i]))
            description['vnfs_chain'] = vnfs_chain
            list_request.append(description)
        file.close()
        with open("request.json", "w") as outfile:
            json.dump(list_request, outfile, indent=4)
        return list_request
    except Exception as e:
        print(e)
        return None

In [107]:
requests = read_request("request10.txt")
requests

[{'memory': 14,
  'bandwidth': 22,
  'cpu': 10,
  'source': 19,
  'destination': 13,
  'vnfs_chain': [2, 8, 3]},
 {'memory': 28,
  'bandwidth': 16,
  'cpu': 21,
  'source': 48,
  'destination': 67,
  'vnfs_chain': [2, 8, 1]},
 {'memory': 13,
  'bandwidth': 13,
  'cpu': 14,
  'source': 82,
  'destination': 20,
  'vnfs_chain': [3, 9, 0, 7]},
 {'memory': 25,
  'bandwidth': 14,
  'cpu': 25,
  'source': 84,
  'destination': 8,
  'vnfs_chain': [3, 4, 3, 0]},
 {'memory': 22,
  'bandwidth': 24,
  'cpu': 25,
  'source': 38,
  'destination': 12,
  'vnfs_chain': [1, 9]},
 {'memory': 37,
  'bandwidth': 20,
  'cpu': 14,
  'source': 97,
  'destination': 30,
  'vnfs_chain': [3, 0, 6, 7]},
 {'memory': 15,
  'bandwidth': 26,
  'cpu': 19,
  'source': 35,
  'destination': 67,
  'vnfs_chain': [3, 4, 9, 1]},
 {'memory': 16,
  'bandwidth': 14,
  'cpu': 15,
  'source': 25,
  'destination': 14,
  'vnfs_chain': [3, 4, 1, 3]},
 {'memory': 9,
  'bandwidth': 19,
  'cpu': 21,
  'source': 77,
  'destination': 93,
 

In [108]:
def shortes_path(graph,source, destination): # return path from source to destination
    visited = [False] * len(graph)
    distance = [float('inf')] * len(graph)
    path = [-1] * len(graph)
    distance[source] = 0
    for i in range(len(graph)):
        min_distance = float('inf')
        min_index = -1
        for j in range(len(graph)):
            if visited[j] == False and distance[j] < min_distance:
                min_distance = distance[j]
                min_index = j
        visited[min_index] = True
        neighbor = graph[min_index].get("neighbor", {})
        for j in neighbor:
            if visited[j] == False and distance[j] > distance[min_index] + neighbor[j]:
                distance[j] = distance[min_index] + neighbor[j]
                path[j] = min_index
    path_result = []
    while destination != -1:
        path_result.append(destination)
        destination = path[destination]
    return path_result[::-1]

In [109]:
path_demo = shortes_path(graph, 10, 5)
path_demo

[10, 15, 5]

In [110]:
def place_vnfs(request, list_server_node):
    nums_vnfs = len(request['vnfs_chain'])
    node_choosen = random.sample(list_server_node, nums_vnfs)
    return node_choosen



In [111]:
place_vnfs(requests[0], list_server_node)

[33, 46, 65]

In [112]:
def define_path_each_request(graph, request, list_server_node,): # trả lại 1 gen gồm đường đi và vị trí cài đặt vnf
    
    node_choosen = place_vnfs(request, list_server_node)
    node_choosen2 = node_choosen.copy()
    source_node = request['source']
    destination_node = request['destination']
    if node_choosen[0] != source_node:
        node_choosen.insert(0, source_node)
    if node_choosen[-1] != destination_node:
        node_choosen.append(destination_node)
    path = shortes_path(graph, node_choosen[0], node_choosen[1])
    for i in range(1,len(node_choosen) - 2):
        sub_path = shortes_path(graph, node_choosen[i], node_choosen[i+1])[1:]
        path += sub_path
    sub_path = shortes_path(graph, node_choosen[-2], node_choosen[-1])[1:]
    return (node_choosen2,path+sub_path)


In [113]:
path_demo_request_0 = define_path_each_request(graph, requests[0], list_server_node)
print(path_demo_request_0)

([33, 75, 17], [19, 5, 15, 20, 33, 43, 40, 46, 49, 56, 75, 56, 49, 46, 40, 36, 35, 29, 23, 17, 9, 13])


In [114]:
def init_individual(graph, list_request, list_server_node): # trả lại 1 cá thể
    individual = []
    for request in list_request:
        individual.append(define_path_each_request(graph, request, list_server_node))
    return individual


In [115]:
individual = init_individual(graph, requests, list_server_node)
with open("individual.json", "w") as outfile:
    json.dump(individual, outfile, indent=4)

In [116]:
def resource_limit(set_request): # trả lại giới hạn tài nguyên dùng để tính toán các ràng buộc 
    cpu_limit = 0
    memory_limit = 0
    bandwidth_limit = 0
    for request in set_request:
        nums_vnf = len(request.get("vnfs_chain"))
        cpu_limit += request.get("cpu") * nums_vnf
        memory_limit += request.get("memory") * (nums_vnf + 1)
        bandwidth_limit += request.get("bandwidth") * (nums_vnf + 1)
    return cpu_limit, memory_limit, bandwidth_limit

In [117]:
def verify_banwidth(individual ,max_bandwidth, requests):# valind bandwidth
    bandwidth_sum = {}
    for index,gen in enumerate(individual):
        path = gen[1]
        for i in range(len(path)-1):
            u = path[i]
            v = path[i+1]
            if (u,v) in bandwidth_sum:
                bandwidth_sum[(u,v)] += requests[index].get("bandwidth")
            else:
                bandwidth_sum[(u,v)] = requests[index].get("bandwidth")
    for key in bandwidth_sum:
        if bandwidth_sum[key] > max_bandwidth:
            return False
    return True

def verify_cpu(individual, max_cpu, requests): # valind cpu
    cpu_sum = {}
    for index,gen in enumerate(individual):
        path = gen[0]
        for node in path:
            if node in cpu_sum:
                cpu_sum[node] += requests[index].get("cpu")
            else:
                cpu_sum[node] = requests[index].get("cpu")
    for key in cpu_sum:
        if cpu_sum[key] > max_cpu:
            return False
    return True

def verify_memory(individual, max_memory, graph, requests): # valind memory
    memory_sum = {}
    for index,gen in enumerate(individual):
        path = gen[1]
        for node in path:
            if graph[node]['is_server'] == False:
                if node in memory_sum:
                    memory_sum[node] += requests[index].get("memory")
                else:
                    memory_sum[node] = requests[index].get("memory")
                # memory_sum[node] = memory_sum.get(node,0) + requests[index].get("memory")
    for key in memory_sum:
        if memory_sum[key] > max_memory:
            return False
    return True

def verify_vnf_each_node(individual, max_install): # valind vnf each server node
    vnf_count = {}
    for gen in individual:
        path = gen[0]
        for node in path:
            vnf_count[node] = vnf_count.get(node,0) + 1
    for key in vnf_count:
        if vnf_count[key] > max_install:
            return False
    return True
    
        
    

In [118]:
cpu_limit, memory_limit, bandwidth_limit = resource_limit(requests)

In [119]:
verify_banwidth(individual, bandwidth_limit, requests)
# caculate_cpu(individual, max_cpu, requests) 
# caculate_memory(individual, max_memory, graph, requests) 
# caculate_vnf_each_node(individual, max_install_each_node)


True

In [120]:
def valid_condition(individual, max_bandwidth, max_cpu, max_memory, max_install, graph, requests):
    if verify_banwidth(individual, max_bandwidth, requests) and verify_cpu(individual, max_cpu, requests) and verify_memory(individual, max_memory, graph, requests): #and caculate_vnf_each_node(individual, max_install):
        return True
    return False

In [121]:
def latency_of_all_requests(graph, individual, max_latency):
    latency = 0
    for gen in individual:
        path = gen[1]
        for i in range(len(path)-1):
            u = path[i]
            v = path[i+1]
            latency += graph[u]["neighbor"][v]
    for gen in individual:
        path = gen[0]
        for i in path:
            latency += graph[i]["delay"]
    return latency / (max_latency*len(individual))
    

In [122]:
latency_demo = latency_of_all_requests(graph, individual, max_latency)
latency_demo


0.3303697595114449

In [123]:
def cost_deploy_server_node(graph, individual, max_cost_active):
    cost = 0
    set_server = set()
    for gen in individual:
        path = gen[0]
        for i in path:
            set_server.add(i)
    for i in set_server:
        cost += graph[i]["active_cost"]
    return cost / max_cost_active


In [124]:
cost_deploy_demo = cost_deploy_server_node(graph, individual, max_cost_active)
cost_deploy_demo

1.0

In [125]:
def cost_install_vnf(individual, max_cost_install, requests,graph):
    cost = 0
    for index,gen in enumerate(individual):
        path = gen[0]
        for i,server_node in enumerate(path):
            cost += graph[server_node]["list_install_cost"][requests[index].get("vnfs_chain")[i]]
    return cost / max_cost_install


In [126]:
cost_install_demo = cost_install_vnf(individual, max_cost_install, requests,graph)
cost_install_demo

0.3361267754495305

In [127]:
def finness_function(individual, graph, requests, max_latency, max_cost_active, max_cost_install):
    return latency_of_all_requests(graph, individual, max_latency) + cost_deploy_server_node(graph, individual, max_cost_active) + cost_install_vnf(individual, max_cost_install, requests,graph)
  


In [128]:
finess_demo = finness_function(individual,graph, requests, max_latency, max_cost_active, max_cost_install)
finess_demo

1.6664965349609755

In [129]:
def generate_population(population_size, graph, requests, list_server_node, bandwidth_limit, cpu_limit, memory_limit, max_install):
    population = []
    for i in range(population_size):
        individual = init_individual(graph, requests, list_server_node)
        if valid_condition(individual, bandwidth_limit, cpu_limit, memory_limit, max_install, graph, requests):
            population.append(individual)
    return population

In [130]:
population = generate_population(50, graph, requests, list_server_node, bandwidth_limit, cpu_limit, memory_limit, max_install_each_node)
with open("population.json", "w") as outfile:
    json.dump(population, outfile, indent=4)

In [131]:
def tournament_selection(population,tournament_size): # return the best individual in tournament
    tournament = random.sample(population, tournament_size)
    tournament.sort(key = lambda x: finness_function(x,graph, requests, max_latency, max_cost_active, max_cost_install))
    return tournament[0]

In [132]:
tournament_selection(population, 5)

[([15, 46, 3],
  [19, 5, 15, 20, 33, 43, 40, 46, 40, 36, 35, 29, 23, 17, 9, 3, 9, 13]),
 ([3, 40, 17],
  [48,
   53,
   54,
   65,
   62,
   60,
   58,
   55,
   52,
   50,
   46,
   40,
   36,
   35,
   29,
   23,
   17,
   9,
   3,
   9,
   17,
   23,
   29,
   35,
   36,
   40,
   36,
   35,
   29,
   23,
   17,
   23,
   29,
   35,
   36,
   40,
   46,
   49,
   56,
   75,
   78,
   66,
   68,
   63,
   67]),
 ([33, 46, 9, 40],
  [82,
   84,
   83,
   74,
   67,
   63,
   68,
   66,
   78,
   75,
   56,
   49,
   46,
   40,
   43,
   33,
   43,
   40,
   46,
   40,
   36,
   35,
   29,
   23,
   17,
   9,
   17,
   23,
   29,
   35,
   36,
   40,
   43,
   33,
   20]),
 ([3, 65, 75, 40],
  [84,
   83,
   74,
   67,
   63,
   68,
   66,
   78,
   75,
   56,
   49,
   46,
   40,
   36,
   35,
   29,
   23,
   17,
   9,
   3,
   9,
   17,
   23,
   29,
   35,
   36,
   40,
   46,
   50,
   52,
   55,
   58,
   60,
   62,
   65,
   62,
   67,
   63,
   68,
   66,
   78,
   75,
   56,
 

In [133]:
def crossover(individual1, individual2, attemp = 100): # return 2 children
    # thực hiện cắt ghép tối đa 100 lần, nếu không tìm được cá thể hợp lệ thì trả lại 2 cá thể ban đầu
    for i in range(attemp):
        point = random.randint(0, len(individual1))
        child1 = individual1[:point] + individual2[point:]
        child2 = individual2[:point] + individual1[point:]
        if valid_condition(child1, bandwidth_limit, cpu_limit, memory_limit, max_install_each_node, graph, requests) and valid_condition(child2, bandwidth_limit, cpu_limit, memory_limit, max_install_each_node, graph, requests):
            return child1, child2
    return individual1, individual2

In [134]:
def mutate(individual, attemp = 100): # thực hiện đột biến tối đa 100 lần, nếu không tìm được cá thể hợp lệ thì trả lại cá thể ban đầu
    if random.random() < 0.1:
        for i in range(attemp):
            point = random.randint(0, len(individual)-1)
            individual[point] = define_path_each_request(graph, requests[point], list_server_node)
            if valid_condition(individual, bandwidth_limit, cpu_limit, memory_limit, max_install_each_node, graph, requests):
                return individual
        return individual
    return individual
   

In [144]:
def genetic_algorithm(num_generation,population_size, tournament_size, elite_rate= 0.5):
    population = generate_population(population_size, graph, requests, list_server_node, bandwidth_limit, cpu_limit, memory_limit, max_install_each_node)
    sorted_population = sorted(population, key = lambda x: finness_function(x,graph, requests, max_latency, max_cost_active, max_cost_install))
    elite_size = int(population_size * elite_rate)
    population = sorted_population[:elite_size]
    for generation in range(num_generation):
        new_population = []
        for i in range(elite_size,population_size):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
        population = sorted(population, key = lambda x: finness_function(x,graph, requests, max_latency, max_cost_active, max_cost_install))
    return population[0]

In [140]:
def cost_install_vnf2(individual, max_cost_install, requests,graph):
    cost = 0
    for index,gen in enumerate(individual):
        path = gen[0]
        for i,server_node in enumerate(path):
            cost += graph[server_node]["list_install_cost"][requests[index].get("vnfs_chain")[i]]
    return cost

In [141]:
def latency_of_all_requests2(graph, individual, max_latency):
    latency = 0
    for gen in individual:
        path = gen[1]
        for i in range(len(path)-1):
            u = path[i]
            v = path[i+1]
            latency += graph[u]["neighbor"][v]
    for gen in individual:
        path = gen[0]
        for i in path:
            latency += graph[i]["delay"]
    return latency 

In [145]:
best_individual = genetic_algorithm(100, 50, 5)
print(best_individual)
print(cost_install_vnf2(best_individual, max_cost_install, requests, graph))
print(latency_of_all_requests2(graph, best_individual, max_latency))

[([40, 33, 46], [19, 28, 40, 43, 33, 43, 40, 46, 40, 36, 35, 29, 23, 17, 9, 13]), ([17, 33, 40], [48, 53, 54, 65, 62, 60, 58, 55, 52, 50, 46, 40, 36, 35, 29, 23, 17, 23, 29, 35, 36, 40, 43, 33, 43, 40, 46, 49, 56, 75, 78, 66, 68, 63, 67]), ([75, 46, 40, 17], [82, 84, 83, 74, 67, 63, 68, 66, 78, 75, 56, 49, 46, 40, 36, 35, 29, 23, 17, 23, 29, 35, 36, 40, 43, 33, 20]), ([75, 17, 46, 40], [84, 83, 74, 67, 63, 68, 66, 78, 75, 56, 49, 46, 40, 36, 35, 29, 23, 17, 23, 29, 35, 36, 40, 46, 40, 36, 35, 29, 23, 17, 12, 7, 8]), ([40, 46], [38, 29, 35, 36, 40, 46, 40, 36, 35, 29, 23, 17, 12]), ([75, 46, 17, 3], [97, 99, 100, 94, 87, 85, 83, 74, 67, 63, 68, 66, 78, 75, 56, 49, 46, 40, 36, 35, 29, 23, 17, 9, 3, 9, 17, 23, 29, 35, 26, 21, 30]), ([33, 32, 40, 46], [35, 36, 40, 43, 33, 43, 40, 36, 35, 29, 23, 17, 9, 13, 32, 13, 9, 17, 23, 29, 35, 36, 40, 46, 49, 56, 75, 78, 66, 68, 63, 67]), ([46, 40, 3, 17], [25, 20, 33, 43, 40, 46, 40, 36, 35, 29, 23, 17, 9, 3, 9, 17, 9, 13, 32, 31, 22, 14]), ([40, 33